<a href="https://colab.research.google.com/github/IKeeso/Project9_data/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stanza
!pip install conllu
!git clone https://github.com/IKeeso/Project9_data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 26.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [2]:
import stanza
from stanza.utils.conll import CoNLL
from conllu import parse
from conllu import TokenList
import re
import os

!ls Project9_data


'autisterna om kvinnor på spektrat.conllu'   expressen.txt
'Autisterna Om kvinnor på spektrat.txt'     'jag upphäver gravitationen.conllu'
 Blogs					    'Jag upphäver gravitationen.txt'
 blogstestfiles				    'konsten att fejka arabiska.conllu'
'en annorlunda barndom.conllu'		    'Konsten att fejka arabiska.txt'
'En annorlunda barndom.txt'		     Project.ipynb
'ett annorlunda liv.conllu'		    'vanliga människor autistens guide till galaxen.conllu'
'Ett annorlunda liv.txt'		    'Vanliga människor autistens guide till galaxen.txt'
 expressen.conllu


In [3]:
stanza.download('sv')
nlp = stanza.Pipeline(lang='sv')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |
| ner       | suc3shuffle        |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [4]:
def tag_and_convert(filename):
  %cd /content/Project9_data
  try:
    with open(filename, 'r', encoding='utf-8') as file:
      text = file.read()
      file.close()
  except:
    text = filename
  doc = nlp(text)
  pathname = filename.split(".")[0]
  conllufilename = f"{pathname}.conllu".lower()
  CoNLL.write_doc2conll(doc, conllufilename)

  return conllufilename

In [10]:
class SemanticAnalyzer:
  def __init__(self, file_path):
    self.file_path = file_path
    self.data = self.parse_conll_u(self.file_path)
    self.PFL_sentence_matches = self.person_first_lang(self.data)
    self.IFL_sentence_matches = self.identity_first_lang(self.data)

  def parse_conll_u(self, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
      data = {}
      sentence = []
      for line in file:
        line = line.strip()
        if line.startswith('# text'):
          sentence = []
        if line.startswith("# sent_id"):
          line = line.split("=")
          key = line[1].strip()
          sentence = []
        elif line == '':
          if sentence:
            data[key] = sentence
        else:
            sentence.append(line.split('\t'))
    return data

  def person_first_lang(self, data):
    PFL_sents = {}
    PFL_regx = re.compile(r"ha|med", re.IGNORECASE)

    for id, sentence in data.items():
      for token in sentence:
        if token[1] == "autism" or token[1].lower() == "autismspektrumtillstånd" or token[1].lower() == "ast" or token[1].lower() == "autismdiagnos":
          if sentence[int(token[0])-2][1] == "och".lower():
            if PFL_regx.match(sentence[int(sentence[int(token[6])-1][int(6)])-1][1]): #Huvudet till huvudet till ordet
              PFL_sents[id] = sentence
          if PFL_regx.match(sentence[int(token[6])][2].lower()): #Ordet innan lemma
            PFL_sents[id] = sentence
    return PFL_sents

  def identity_first_lang(self, data):
    IFL_sents = {}
    IFL_regx = re.compile(r"^.*(autistisk|autistiska)", re.IGNORECASE)
    for id, sentence in data.items():
      for token in sentence:

        #Alla ord som är ifl oavsett kontext
        if token[1] == "autist" or token[1] == "autisten" or token[1] == "autister":
          IFL_sents[id] = sentence

        #på spektrat är ifl
        if token[1] == "spektrumet" or token[1] == "autismspektrumet" or token[1] == "spektrat":
          if sentence[int(token[6])][1].lower() == "på": #Ordets huvud
            IFL_sents[id] = sentence
            continue

        #Om ordet är autistiska/autistisk låt användaren manuellt kolla om det är ifl
        if IFL_regx.match(token[1].lower()) and token[1] != "autistiskt":
          if sentence[int(token[0])-1][7] in ("nsubj:pass", "nsubj"):
            print(sentence[int(token[0])]) #continue HERREEEEEEE
          test = ' '.join(sublist[1] for sublist in sentence)
          print(test)
          while True:
            question = input("Is this ifl (y/n): ")
            if question.lower() not in ('y', 'n'):
                print("Not an appropriate choice.")
                print(test)
            else:
              if question.lower() == "y":
                IFL_sents[id] = sentence
                break
              if question.lower() == "n":
                break

    return IFL_sents

####

def main(file, lang):
  analyzer = SemanticAnalyzer(file_path= file)
  if lang.lower() == "pfl":
    matches = analyzer.PFL_sentence_matches
  elif lang.lower() == "ifl":
    matches = analyzer.IFL_sentence_matches
  else:
    print("Invalid language choice. Please choose either 'PFL' or 'IFL'.")
    return

  for id, sentence in matches.items():
    print(f"Sentence ID: {int(id) + 1} | {' '.join(token[1] for token in sentence)}")
    print("")

  print(f"Total {lang} matching sentences: {len(matches)}")

if __name__ == "__main__":
    #file_path = input("Enter the file path: ").lower()
    #if ".conllu" not in file_path:
      #tag_and_convert(file_path)
      #file_path = tag_and_convert(file_path)
    #lang = input("Enter the language (PFL or IFL): ")

    root_path = r'/content/Project9_data/blogstestfiles'
    for filename in os.listdir(root_path):
      filepath = os.path.join(root_path, filename)
      print(f"Testing {filename}:")
      main(filepath, "ifl")
      print("\n")
      main(filepath, "pfl")
      print("--------------------")

Testing så kan en person med autism använda sina styrkor.conllu:
Sentence ID: 3 | Jag tycker att det är positivt att styrkorna hos oss med autism uppmärksammas , men samtidigt tycker jag inte om att omgivningen ofta har åsikter om på vilket sätt en autist borde använda sina styrkor .

Sentence ID: 6 | Om en autist använder sina styrkor i yrkeslivet anses hen ofta använda dem på ”rätt sätt” , men om en autist med styrkor och talanger är sjukpensionär och samtidigt kanske har boendestöd eller bor på ett gruppboende anses hen ofta använda dessa på fel sätt .

Sentence ID: 32 | På grund av den funktionalistiska människosynen anses deltagandet i arbetslivet vara det enda sättet för autisten att använda sina styrkor på , och detta oavsett autistens egna drömmar , ambitioner och målsättningar .

Sentence ID: 34 | Låt oss autister välja hur vi vill nyttja vår potential !

Sentence ID: 35 | När en autist mår bra och trivs med sin vardag använder hen sina styrkor på rätt sätt , punkt slut .

Sen

KeyboardInterrupt: Interrupted by user

In [6]:
"""
import os
from google.colab import files

# Define the directory path
directory = '/content/Project9_data/Blogs'
outdir = '/content/Project9_data/Testfiles'

# Iterate through files in the directory
for folder in os.listdir(directory):
  for filename in os.listdir(os.path.join(directory, folder)):
    folderpath = os.path.join(directory, folder)
    filepath = os.path.join(folderpath, filename)
    print("Converting and downloading: ", filename)
    files.download(os.path.join(outdir, tag_and_convert(filepath)))
"""

'\nimport os\nfrom google.colab import files\n\n# Define the directory path\ndirectory = \'/content/Project9_data/Blogs\'\noutdir = \'/content/Project9_data/Testfiles\'\n\n# Iterate through files in the directory\nfor folder in os.listdir(directory):\n  for filename in os.listdir(os.path.join(directory, folder)):\n    folderpath = os.path.join(directory, folder)\n    filepath = os.path.join(folderpath, filename)\n    print("Converting and downloading: ", filename)\n    files.download(os.path.join(outdir, tag_and_convert(filepath)))\n'